# Install requirements

In [1]:
%%capture
!pip install pandas-profiling==3.5.0

# Sample Comparison Report

In [2]:
from typing import Dict
import time
import os
from pathlib import Path
import pandas as pd
from pathlib import Path
from pandas_profiling import ProfileReport

from ydata.metadata import Metadata
from ydata.dataset import Dataset

from common.utils import load_dataframe, get_model_class

import sdv

from common.config import *

datasets_config = get_datsets_config()
models_config = get_models_config()
analysis_config = get_analysis_config()

for dataset_name in datasets_config.keys():
    print(f'# Dataset: {dataset_name} ')
    dataset = load_dataframe(dataset_name, split='train')
    metadata = Metadata.load(str(Path(DATASET_PATH) / f'{dataset_name}_train.metadata.pkl'))
    for model_name, model_info in models_config.items():
        if not model_info.get('enabled', True):
            continue
            
        report_path = Path(ANALYSIS_REPORTS_PATH) / f"{dataset_name}_{model_name}_report.html"
        if not os.path.isfile(report_path):
            try:
                holdout_df = pd.read_csv(Path(DATASET_PATH) / f'{dataset_name}_holdout.csv')
                sample_df = pd.read_csv(Path(SAMPLES_PATH) / f'{model_name}_{dataset_name}_sample.csv')

                holdout_profile = ProfileReport(holdout_df, title=f'Holdout - {dataset_name}')
                sample_profile  = ProfileReport(sample_df, title=f'Sample - {dataset_name} - Model {model_name}')
                comparison = holdout_profile.compare(sample_profile)

                comparison.to_file(report_path)
            except Exception as e:
                print(f'Could not generate the comparison report for dataset {dataset_name} and model {model_name}: {e}')
        else:
            print(f'Report already exist for dataset {dataset_name} and model {model_name}')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]